In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa as tsa
from statsmodels.tsa.arima_process import arma_generate_sample
import matplotlib.pyplot as plt 
from statsmodels.graphics.api import qqplot
from statsmodels.tsa.stattools import adfuller, zivot_andrews
%matplotlib inline
from scipy import stats

# Evaluación parcial

**Instrucciones:**

**¿Qué archivos necesito para realizar esta evaluación?** Usted necesita tres archivos: este notebook y 2 archivos csv (están en el gitbub del curso carpteta examen, busque los dos con sus apellidos terminando en "_1" y "_2"). Guarde esos archivos en su computadora.MANTENGA EL NOMBRE DE LOS ARCHIVOS CSV.

**¿Dónde realizo la evaluación?** Desarrolle la evalución completamente en este notebook. NO use notebook adicional ni ningún otro software.

**¿Qué debo entregar?**. Usted deberá entregar este notebook. LLamelo con sus apellidos. Por ejemplo si su nombre es Leonardo Sánchez, el notebok se llamará: evaluacion_parcial_Leonardo_Sanchez

**¿Dónde debo subir el notebook?** Subirlo en el ambiente creado en MS Team del curso de econometría III.


_________
________

# Problema (55 puntos)

En esta sección usted deberá seleccionar el modelo ARMA que mejor se ajusta a los datos. 

## Importar los datos (5 puntos)

Considere que la serie de datos disponible tiene frecuencia mensual, y comienza desde enero 2001.

In [ ]:
# aquí su código

## Muestre las primeras 5 observaciones de los datos (2 puntos)

In [ ]:
# aquí su código

## Grafique la serie (5 puntos)

In [ ]:
# aquí su código

## Grafique las ACF y PACF (5 puntos)

In [ ]:
# aquí su código

## Qué tipo de proceso le sugieren estas funciones? (3 puntos)

aquí su comentario

## Usando los criterios de Akaike (AIC) y Bayesin (BIC), seleccione  indique que proceso esta representado por los datos (5 puntos)

In [ ]:
# aquí su código

aquí su comentario

## Estime el proceso sugerido por el literal anterior (5 puntos)

In [ ]:
# Cambie el código apropiadamente
model = sm.tsa.SARIMAX(variable , order=(0,0,0), trend=None)

# Estimar los parametros del modelo
results = model.fit()

print(results.summary())

## Realice el pronóstico de los valores que tomaría la variable en los siguientes tres meses (5 puntos)

In [ ]:
# aquí su código

## Validación cruzada (20 puntos)

A continuación deberá validar si el modelo estimado en el sección 2.7 es mejor que alguna alternativa sugerida. Especificamente, su profesor considera que el los datos que usted posee provienen de un proceso AR(1) 

$y_t= \beta_0 + \beta_1 y_{t-1}+ e_t$

**Primero (12 puntos).** Haga la validación cruzada del modelo estimado en 2.7. Use 3 pronósticos y el 80\% de los datos como training data. Al final deberá mostrar el error cuadrático medio para cada uno de estos tres pronósticos relacionados al modelo 2.7. A continuación el código que le permite desarrollar eso. Deberá ajustarlo apropiadamente.

In [ ]:
# Establecer el numero de pronosticos
nforecasts = 3
forecasts = {}

# Obtener el numero de observaciones del training initial
nobs = len(z)
n_init_training = int(nobs * 0.56)

# Crear el model para estimar con la muestra del initial training sample
init_training_z = z.iloc[:n_init_training]
mod = sm.tsa.SARIMAX(training_z, order=(1, 0, 0), trend=None)
res = mod.fit()

# Guardar los pronosticos inicialest
forecasts[training_z.index[-1]] = res.forecast(steps=nforecasts)

# Realizar el procedimiento al resto de la muestra
for t in range(n_init_training, nobs):
    # Actualizar los resultados mediante el agregar una observacion adicional
    updated_z = z.iloc[t:t+1]
    res = res.append(updated_z, refit=False)

    # Guardar los nuevos resultados
    forecasts[updated_z.index[0]] = res.forecast(steps=nforecasts)

# Combinar todos los pronosticos dentro de un dataframe
forecasts = pd.concat(forecasts, axis=1)

print(forecasts.iloc[:10, :7])
# print(forecasts)

In [ ]:
# Construir los errores de pronosticos
forecast_errors = forecasts.apply(lambda column: z - column).reindex(forecasts.index)

print(forecast_errors.iloc[:10, :5])

In [ ]:
# Reindex los pronosticos en funcion de los horizonte
def flatten(column):
    return column.dropna().reset_index(drop=True)

forecast_horizon = forecast_errors.apply(flatten)
forecast_horizon.index = (forecast_horizon.index + 1).rename('horizon')

print(forecast_horizon.iloc[:3, :5])

In [ ]:
# Calcular la raiz del mean square error
rmse = (forecast_horizon**2).mean(axis=1)**0.5

print(rmse)

**Segundo (6 puntos).** Repita la pregunta 2.7 pero considere que el proceso que siguen sus datos es el indicado por su profesor. Una vez hecho esto, repita la validación cruzada pero con este nuevo modelo.

In [ ]:
# Crear el model para estimar con la muestra del initial training sample
init_training_z = z.iloc[:n_init_training]
mod = sm.tsa.SARIMAX(training_z, order=(1, 0, 0), trend=None)
res = mod.fit()

# Guardar los pronosticos inicialest
forecasts[training_z.index[-1]] = res.forecast(steps=nforecasts)

# Realizar el procedimiento al resto de la muestra
for t in range(n_init_training, nobs):
    # Actualizar los resultados mediante el agregar una observacion adicional
    updated_z = z.iloc[t:t+1]
    res = res.append(updated_z, refit=False)

    # Guardar los nuevos resultados
    forecasts[updated_z.index[0]] = res.forecast(steps=nforecasts)

# Combinar todos los pronosticos dentro de un dataframe
forecasts = pd.concat(forecasts, axis=1)

print(forecasts.iloc[:10, :7])
# print(forecasts)

In [ ]:
# Construir los errores de pronosticos
forecast_errors = forecasts.apply(lambda column: z - column).reindex(forecasts.index)

print(forecast_errors.iloc[:10, :5])

In [ ]:
# Reindex los pronosticos en funcion de los horizonte
def flatten(column):
    return column.dropna().reset_index(drop=True)

forecast_horizon = forecast_errors.apply(flatten)
forecast_horizon.index = (forecast_horizon.index + 1).rename('horizon')

print(forecast_horizon.iloc[:3, :5])

In [ ]:
# Calcular la raiz del error cuadrático medio
rmse = (forecast_horizon**2).mean(axis=1)**0.5

print(rmse)

**Tercero (2 puntos).** A la luz de estos resultados, ¿qué modelo arroja el menor error cuadrático medio?

aquí su respuesta

_____
_____

# Problema (45 puntos)

En esta sección usted deberá especificar si la serie de datos tiene una tendencia determinística, o estocástica con o sin drfit, o si el proceso es estacionario. Además deberá indicar si la serie presenta algún quiebre en la media o en la tendencia.

## Importar la serie apellidos_2 (5 puntos)

In [ ]:
# aquí su código

## Grafique la serie (5 puntos)

In [ ]:
# aquí su código

Para contestar las siguientes dos preguntas necesitará la siguiente función

_____

In [ ]:
def zivot(timeseries,exog):
    print ('Results of Zivot-Andrews Test:')
    dftest = zivot_andrews(timeseries, trim=0.10, maxlag=None,regression=exog,autolag=None)
    dfoutput = pd.Series(dftest[0:2], index=['Test Statistic','p-value'])
    for key,value in dftest[2].items():
        dfoutput['Critical Value (%s)'%key] = value
    #dfoutput['Number of lags']=pd.Series(dftest[3:4])
    dfoutput['break']=pd.Series(dftest[4])
    print (dfoutput)

## Evalue si la serie presenta un quiebre en la media (5 puntos)

In [ ]:
# aquí su código

## Evalue si la serie presente un quiebre en la tendencia (5 puntos)

In [ ]:
# aquí su código

## Con base en los resultados de la prueba de Zivot y Andrews, ¿qué puede concluir? (5 puntos)

aquí su respuesta

____


In [ ]:
Para contestar la siguiente pregunta necesitará la siguiente función

In [ ]:
def adf_test(timeseries,exog):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, maxlag=5,regression=exog,autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

## ¿Qué tipo de tendencia exhibe la serie? Justifique su respuesta siguiendo detalladamente los pasos que presenta Enders (1995), visto en clases (20 puntos) 

In [ ]:
# aquí su código

Su conclusión aquí